# 📌 What This Demo Covers

This demo introduces AI agent building with LangChain’s legacy AgentExecutor, a great starting point for understanding basic tool use and reasoning loops. But for complex, stateful, and production-grade agents, LangChain now recommends LangGraph, which offers finer control, richer workflows, and more reliable orchestration.

Flow:

1. required package installations
2. Authorization to access API end-points
3. setting environment variables
4. building tools
5. checking tool-calling functionality of llms
6. building agents
7. testing agent
8. highlighting the limitation --> leads to context aware agent


## Install OpenAI, and LangChain dependencies

In [1]:
!pip install -q langchain==0.3.14
!pip install -q langchain-openai==0.3.0
!pip install -q langchain-community==0.3.14

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.63 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [2]:
!pip install -q markitdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 890.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.4 MB/s eta 0:00:00


## Enter Open AI API Key

In [3]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [4]:
TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

Enter Tavily Search API Key: ··········


## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [5]:
WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

Enter WeatherAPI API Key: ··········


## Setup Environment Variables

In [6]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Create Tools

Here we create two custom tools which are wrappers on top of the [Tavily API](https://tavily.com/#api) and [WeatherAPI](https://www.weatherapi.com/)

- Web Search tool with information extraction
- Weather tool

In [7]:
from langchain_core.tools import tool
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import requests
import json
from warnings import filterwarnings
filterwarnings('ignore')

tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_answer=False,
                                  include_raw_content=True)
# certain websites won't let you crawl them unless you specify a user-agent
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links."""
    print('Calling web search tool')
    results = tavily_tool.invoke(query)
    docs = []

    def extract_content(url):
        """Helper function to extract content from a URL."""
        extracted_info = md.convert(url)
        text_title = extracted_info.title.strip()
        text_content = extracted_info.text_content.strip()
        return text_title + '\n' + text_content
    # parallelize execution of different urls
    with ThreadPoolExecutor() as executor:
        for result in tqdm(results):
            try:
                future = executor.submit(extract_content, result['url'])
                # Wait for up to 15 seconds for the task to complete
                content = future.result(timeout=15)
                docs.append(content)
            except TimeoutError:
                print(f"Extraction timed out for url: {result['url']}")
            except Exception as e:
                print(f"Error extracting from url: {result['url']} - {e}")

    return docs


@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather of the queried location."""
    print('Calling weather tool')
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

## Test Tool Calling with LLM

In [8]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]

chatgpt_with_tools = chatgpt.bind_tools(tools)

In [13]:
prompt = "Get details of Microsoft's earnings call Q1 2025"
response = chatgpt_with_tools.invoke(prompt)
response.tool_calls

[{'name': 'search_web_extract_info',
  'args': {'query': 'Microsoft earnings call Q1 2025 details'},
  'id': 'call_bggla7gfizNVIOy7pDcTzAxS',
  'type': 'tool_call'}]

In [10]:
prompt = "how is the weather in Hyderabad today"
response = chatgpt_with_tools.invoke(prompt)
response.tool_calls

[{'name': 'get_weather',
  'args': {'query': 'Hyderabad'},
  'id': 'call_H6gZPpopgzmRcCowbsbCrllF',
  'type': 'tool_call'}]

## Build and Test AI Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent to bind the tools to the agent with a prompt. We will also add in the capability to store historical conversations as memory

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """Act as a helpful assistant.
                You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                Repeat till you get to the answer for the given user query.

                Use the following workflow format:
                  Question: the input task you must solve
                  Thought: you should always think about what to do
                  Action: the action to take which can be any of the following:
                            - break it into smaller steps if needed
                            - see if you can answer the given task with your trained knowledge
                            - call the most relevant tools at your disposal mentioned below in case you need more information
                  Action Input: the input to the action
                  Observation: the result of the action
                  ... (this Thought/Action/Action Input/Observation can repeat N times)
                  Thought: I now know the final answer
                  Final Answer: the final answer to the original input question

                Tools at your disposal to perform tasks as needed:
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
             """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Act as a helpful assistant.\n                You run in a loop of Thought, Action, PAUSE, Observation.\n                At the end of the loop, you output an Answer.\n                Use Thought to describe your thoughts about the question you have been asked.\n                Use Action to run one of the actions available to you - then return PAUSE.\n                Observation will be the result of running those actions.\n                Repeat till you get to the answer for the given user query.\n\n                Use the following workflow format:\n                  Question: the input task you must solve\n                  Thought: you should always think about what to do\n                  Action: the action to take which can be any of the following:\n                            - break it into smaller steps if needed\n                            - see if you can

# 🔍 Initializing the Agent

At this stage, we bring together the LLM, the prompt, and the tools to create the agent.
The key idea to remember is this:

1. The agent decides what to do.
2. The AgentExecutor decides to actually do it.

REMEMBER The agent itself never executes tools.
It only produces the plan — the choice of tool and the arguments.
Execution, error handling, and looping are all handled by the executor.

Note that we are passing in the model `chatgpt`, not `chatgpt_with_tools`.

That is because `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

This should ideally be used with an LLM which supports tool \ function calling

In [19]:
from langchain.agents import create_tool_calling_agent

chatgpt = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [search_web_extract_info, get_weather]
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)

Finally, we combine the `agent` (the brains) with the `tools` inside the `AgentExecutor` (which will repeatedly call the agent and execute tools).

In [20]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               early_stopping_method='force',
                               max_iterations=10)

# No access to latest information

In [21]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = chatgpt.invoke(query)
print(response.content)

As of my last update in October 2023, I don't have access to real-time data or specific earnings call transcripts, including Nvidia's Q4 2024 earnings call. However, I can provide a general outline of what key points are typically discussed in such earnings calls:

1. **Financial Performance**: Overview of revenue, net income, and earnings per share compared to previous quarters and year-over-year performance.

2. **Segment Performance**: Breakdown of performance across different business segments, such as Gaming, Data Center, Professional Visualization, and Automotive.

3. **Market Trends**: Discussion on market conditions, demand for GPUs, and trends in AI, gaming, and data center markets.

4. **Product Updates**: Information on new product launches, advancements in technology, and updates on existing product lines.

5. **Guidance**: Forward-looking statements regarding expected performance in the next quarter or fiscal year, including revenue projections and market outlook.

6. **St

In [22]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})

Calling web search tool


 60%|██████    | 3/5 [00:01<00:01,  1.64it/s]

Error extracting from url: https://nvidianews.nvidia.com/_gallery/download_pdf/65d669a33d63329bbf62672a/ - File conversion failed after 3 attempts:
 - PdfConverter threw MissingDependencyException with message: PdfConverter recognized the input as a potential .pdf file, but the dependencies needed to read .pdf files have not been installed. To resolve this error, include the optional dependency [pdf] or [all] when installing MarkItDown. For example:

* pip install markitdown[pdf]
* pip install markitdown[all]
* pip install markitdown[pdf, ...]
* etc.
 - PlainTextConverter threw UnicodeDecodeError with message: 'utf-8' codec can't decode byte 0xfe in position 28: invalid start byte
 - PdfConverter threw MissingDependencyException with message: PdfConverter recognized the input as a potential .pdf file, but the dependencies needed to read .pdf files have not been installed. To resolve this error, include the optional dependency [pdf] or [all] when installing MarkItDown. For example:

* p

100%|██████████| 5/5 [00:02<00:00,  2.22it/s]


In [23]:
response

{'query': "Summarize the key points discussed in Nvidia's Q4 2024 earnings call",
 'output': "Thought: I need to summarize the key points from Nvidia's Q4 2024 earnings call based on the information retrieved. The earnings call likely covered financial performance, strategic initiatives, and market outlook.\n\nAction: I will extract the relevant information from the search results to summarize the key points discussed in the earnings call.\n\nObservation: The key points from Nvidia's Q4 2024 earnings call include:\n\n1. **Record Financial Performance**: Nvidia reported a revenue of $22.1 billion for Q4 2024, a 22% increase from the previous quarter and a 265% increase year-over-year. The earnings per share (EPS) were $4.93 (GAAP) and $5.16 (Non-GAAP), reflecting significant growth.\n\n2. **Data Center Revenue Growth**: Data center revenue reached a record $18.4 billion, with approximately 40% attributed to AI inference. This segment has seen substantial growth, tripling over the fiscal

In [24]:
from IPython.display import display, Markdown

display(Markdown(response['output']))

Thought: I need to summarize the key points from Nvidia's Q4 2024 earnings call based on the information retrieved. The earnings call likely covered financial performance, strategic initiatives, and market outlook.

Action: I will extract the relevant information from the search results to summarize the key points discussed in the earnings call.

Observation: The key points from Nvidia's Q4 2024 earnings call include:

1. **Record Financial Performance**: Nvidia reported a revenue of $22.1 billion for Q4 2024, a 22% increase from the previous quarter and a 265% increase year-over-year. The earnings per share (EPS) were $4.93 (GAAP) and $5.16 (Non-GAAP), reflecting significant growth.

2. **Data Center Revenue Growth**: Data center revenue reached a record $18.4 billion, with approximately 40% attributed to AI inference. This segment has seen substantial growth, tripling over the fiscal year to $47.5 billion.

3. **AI and Software Opportunities**: Nvidia's AI enterprise software is gaining traction, with an annualized revenue run rate of $1 billion. The company is positioning itself as a leader in AI infrastructure, with a focus on generative AI applications across various industries.

4. **Challenges in China**: Revenue from China declined significantly due to U.S. export control regulations. However, Nvidia is adapting by offering alternative products that do not require licenses for the Chinese market.

5. **Positive Market Sentiment**: Following the earnings announcement, Nvidia's stock surged by over 8%, indicating strong investor confidence in the company's growth trajectory and future prospects.

Thought: I now know the final answer.

Final Answer: Nvidia's Q4 2024 earnings call highlighted record revenue of $22.1 billion, driven by a significant increase in data center revenue, particularly from AI applications. The company is expanding its AI enterprise software offerings and adapting to challenges in the Chinese market. Positive market sentiment was reflected in a notable stock price increase following the earnings announcement.

In [25]:
query = """Summarize the key points discussed in Intel's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling web search tool


 80%|████████  | 4/5 [00:03<00:00,  1.40it/s]

Error extracting from url: https://download.intel.com/newsroom/2025/c8e6h3a2/intel-q4-2024y-earnings.pdf - File conversion failed after 1 attempts:
 - PdfConverter threw MissingDependencyException with message: PdfConverter recognized the input as a potential .pdf file, but the dependencies needed to read .pdf files have not been installed. To resolve this error, include the optional dependency [pdf] or [all] when installing MarkItDown. For example:

* pip install markitdown[pdf]
* pip install markitdown[all]
* pip install markitdown[pdf, ...]
* etc.



100%|██████████| 5/5 [00:04<00:00,  1.23it/s]


**Thought:** I need to summarize the key points from Intel's Q4 2024 earnings call, focusing on financial performance, guidance, and any significant developments mentioned during the call.

**Action:** I will extract the relevant information from the search results regarding Intel's Q4 2024 earnings call.

**Observation:** The key points from Intel's Q4 2024 earnings call include:

1. **Financial Performance:**
   - Q4 revenue was $14.3 billion, a 7% decrease year-over-year but a 7% increase sequentially.
   - Non-GAAP gross margin was 42.1%, exceeding guidance by 260 basis points.
   - Q4 earnings per share (EPS) was $0.13, slightly above the expected $0.12.
   - Full-year revenue for 2024 was $53.1 billion, down 2.1% year-over-year.
   - Full-year EPS was -$0.13, reflecting significant losses.

2. **Guidance:**
   - For Q1 2025, Intel expects revenue between $11.7 billion and $12.7 billion, with a gross margin of approximately 36%.
   - EPS guidance for Q1 2025 is breakeven on a non-GAAP basis.

3. **Business Segments:**
   - Intel Foundry revenue was $4.5 billion, up 3% sequentially but still facing significant operating losses.
   - Mobileye and Altera reported revenues of $490 million and $429 million, respectively, showing slight growth.

4. **Strategic Developments:**
   - Intel is focusing on AI PC CPUs and plans to ship over 100 million systems by the end of 2025.
   - The launch of the Panther Lake CPU is on track for the second half of 2025.
   - The company is facing increased competition in the AI PC market and is not yet a significant player in the cloud-based AI data center market.

5. **Leadership Changes:**
   - Following the resignation of CEO Pat Gelsinger, the company is now led by interim co-CEOs Michelle Johnston Holthaus and David Zinsner.

6. **Market Challenges:**
   - Intel is experiencing pressure on gross margins due to product mix and cost structure, with expectations of continued challenges throughout 2025.

**Thought:** I now have a comprehensive summary of the key points discussed in Intel's Q4 2024 earnings call.

**Final Answer:** Intel's Q4 2024 earnings call highlighted a revenue of $14.3 billion (down 7% YoY), a non-GAAP gross margin of 42.1%, and an EPS of $0.13. For Q1 2025, guidance is set at $11.7-$12.7 billion in revenue with breakeven EPS. The company is focusing on AI CPUs and plans to launch Panther Lake in late 2025, while facing increased competition and leadership changes following CEO Pat Gelsinger's resignation.

In [27]:
query = """which company's future outlook looks to be better?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling web search tool


  0%|          | 0/5 [00:00<?, ?it/s]

Error extracting from url: https://www.bloomberg.com/graphics/2023-investment-outlooks/ - 403 Client Error: Forbidden for url: https://www.bloomberg.com/graphics/2023-investment-outlooks/


 40%|████      | 2/5 [00:00<00:00,  5.08it/s]

Error extracting from url: https://www.weforum.org/publications/the-future-of-jobs-report-2023/ - 403 Client Error: Forbidden for url: https://www.weforum.org/publications/the-future-of-jobs-report-2023/


 80%|████████  | 4/5 [00:15<00:04,  4.51s/it]

Extraction timed out for url: https://www.mckinsey.com/capabilities/tech-and-ai/our-insights/the-top-trends-in-tech-2023


100%|██████████| 5/5 [00:15<00:00,  3.16s/it]


KeyboardInterrupt: 

In [29]:
query = """how is the weather in Hyderabad today?
           show detailed statistics
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling weather tool


Thought: I have received the weather data for Hyderabad. Now, I will organize this information into detailed statistics for clarity.

Final Answer: 
Here are the detailed weather statistics for Hyderabad today:

- **Location**: Hyderabad, Telangana, India
- **Temperature**: 
  - Celsius: 23.3°C
  - Fahrenheit: 73.9°F
- **Condition**: Mist
- **Humidity**: 65%
- **Wind**: 
  - Speed: 6.3 mph (10.1 kph)
  - Direction: East (80°)
- **Pressure**: 
  - 1020.0 mb (30.12 in)
- **Precipitation**: 
  - 0.0 mm (0.0 in)
- **Visibility**: 
  - 3.0 km (1.0 miles)
- **Feels Like**: 
  - Celsius: 25.0°C
  - Fahrenheit: 77.0°F
- **Wind Chill**: 
  - Celsius: 24.4°C
  - Fahrenheit: 75.9°F
- **Heat Index**: 
  - Celsius: 25.7°C
  - Fahrenheit: 78.2°F
- **Dew Point**: 
  - Celsius: 15.3°C
  - Fahrenheit: 59.5°F
- **UV Index**: 5.6
- **Gusts**: 
  - Speed: 7.2 mph (11.6 kph)

This information provides a comprehensive overview of the current weather conditions in Hyderabad.

In [30]:
query = """how is the weather in Bangalore today?
           show detailed statistics
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling weather tool


Thought: I have received the weather data for Bangalore. Now, I will organize this information into detailed statistics for clarity.

Final Answer: 
The current weather in Bangalore is as follows:

- **Location**: Bangalore, Karnataka, India
- **Temperature**: 21.1°C (70.0°F)
- **Condition**: Mist
- **Humidity**: 73%
- **Wind**: 
  - Speed: 14.1 mph (22.7 kph)
  - Direction: East (82°)
- **Pressure**: 
  - 1020.0 mb (30.12 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Cloud Cover**: 50%
- **Visibility**: 2.5 km (1.0 miles)
- **UV Index**: 6.0
- **Feels Like**: 21.1°C (70.0°F)
- **Dew Point**: 13.4°C (56.1°F)
- **Wind Chill**: 21.9°C (71.4°F)
- **Heat Index**: 24.5°C (76.2°F)
- **Gusts**: 
  - Speed: 16.2 mph (26.1 kph)

This data was last updated on November 27, 2025, at 10:45 AM local time.

In [31]:
query = """which city is hotter?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Question: which city is hotter?
Thought: To determine which city is hotter, I need to know the specific cities in question. Without that information, I cannot provide an accurate answer.
Action: I will ask the user to specify the cities they are interested in comparing.
Action Input: "Please specify the two cities you would like to compare in terms of temperature."
Observation: I have prompted the user for the necessary information to proceed with the comparison.

# 🧠 Adding Memory Next

The agent is already performing well, but there’s one limitation: it has no recollection of past interactions.
To make it truly conversational, we’ll introduce user-session memory so the agent can retain context across turns.

We’ll dive deeper into how this memory works — and how to design it properly — in the next notebook.